In [3]:
#### LIBRARIES AND FUNCTIONS
using Plots
using JLD2
using Statistics
using Random
using LinearAlgebra
using StatsBase
using Integrals
using GenericSchur
using Interpolations
using FiniteDifferences
using LaTeXStrings

function Expanding_Map_Integrate(x0,p)
    a,b,c,σ,tr,M = p
    x = zeros(M+tr)
    xOld = x0;
    for i in 1:M+tr
        x[i] = mod( a*xOld - b*sin(6*xOld) + c*cos(3*xOld) + σ*randn() , 2*π)
        xOld = x[i]
    end
    return x[1+tr:end]
end

function return_obs(x)
    return [ cos(x) , cos(2*x), cos(3*x) , cos(4*x), cos(5*x),
             sin(x), sin(2*x), sin(3*x),sin(4*x),sin(5*x), 
             1.0 ./(2+sin(2*x)) , cos(atan(3*sin(x)))/sin(atan(3)), atan(20*sin(2*x))/ atan(20), (1/2+1/2*sin(2*x)) ./ (2+cos(10*x)), (x-pi) .^2
    ] 
end

return_obs (generic function with 1 method)

In [2]:
# Parameters of the map
a = 3; b = 0.4 ; c = 0.08; σ = 0
tr = 500 # Transient to remove
M = 10^7 # Length of trajectory
x0 = 2*pi*rand() # Initial condition
p = (a,b,c,σ,tr,M) 
x = Expanding_Map_Integrate(x0,p);
avg = mean( stack( return_obs.(x) ,dims=1) ,dims=1);

In [3]:
n_Traj = 5*10^6;
tr = 0; M = 50
p = (a,b,c,σ,tr,M) 
amplitudes = [0.02,0.03,0.04,0.05,0.06,0.07];

In [4]:
Data = Array{Matrix{Float64}}(undef, length(amplitudes))

for (ii,ε) in enumerate(amplitudes)
    ICs_plus = mod.(x .+ ε,2*π);
    G_plus = zeros(M,size( return_obs(x[1]) )[1])

    for i in 1:n_Traj
        y = Expanding_Map_Integrate(ICs_plus[i],p);
        g = stack( return_obs.(y) ,dims=1)
        G_plus = G_plus .+ g ./n_Traj
    end
    ICs_plus = nothing; GC.gc()

    ICs_minus = mod.(x .- ε,2*π);
    G_minus = zeros(M,size( return_obs(x[1]) )[1])
    for i in 1:n_Traj
        y = Expanding_Map_Integrate(ICs_minus[i],p);
        g = stack( return_obs.(y) ,dims=1)
        G_minus = G_minus .+ g ./n_Traj
    end
    ICs_minus = nothing; GC.gc()
    G = ( G_plus .- G_minus ) ./(2*ε);

    Data[ii] = G
end

In [5]:
jldsave("Data_GreenFunctions.jld2"; Data,p,n_Traj,amplitudes,return_obs)